In [15]:
import pickle
import torch
from src.model import Transformer, TransformerConfig
import json

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [63]:
model_folder = "run_data"#"models"

with open(model_folder+"/config.json", "r") as f:
    settings = json.loads(f.read())

with open(model_folder+"/word_data.json", "r") as f:
    word_data = json.loads(f.read())

word_to_index = {k:int(v) for k, v in word_data["word_to_index"].items()}
index_to_word = {int(k):v for k, v in word_data["index_to_word"].items()}

In [42]:
# import torch
# import seaborn as sns

# initial_lr = 0.001
# max_lr = 0.004
# min_lr = 0.0001
# optimizer = torch.optim.Adam([torch.Tensor([[1, 2]]), torch.Tensor([[1]])], lr=initial_lr)

# total_epochs = 200
# warmup_steps = 40
# scheduler1 = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=initial_lr/max_lr, total_iters=warmup_steps)
# scheduler2 = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = total_epochs-warmup_steps, eta_min = min_lr)
# scheduler = torch.optim.lr_scheduler.SequentialLR(optimizer, schedulers=[scheduler1, scheduler2], milestones=[warmup_steps])

# lrs = []
# for i in range(total_epochs):
#     lr = scheduler.optimizer.param_groups[0]['lr']
#     lrs.append(lr)
#     scheduler.step()

# sns.lineplot(lrs)

In [64]:
config = TransformerConfig(vocab_size=settings["vocabulary_size"], max_input_length=settings["max_input_length"], num_heads=settings["num_heads"], num_blocks=settings["num_blocks"], embedding_size=settings["embedding_size"])
transformer = Transformer(config)
transformer.load_state_dict(torch.load(model_folder+"/model.pt", map_location=torch.device(device)))
transformer.to(device)
transformer.train()
print("Model loaded")

Model loaded


In [74]:
def generate_next_token(tokens=None, sample=False):
    if tokens == None:
        tokens = [len(index_to_word)-1]
    x = torch.LongTensor([tokens]).to(device)
    with torch.no_grad():
        y = transformer(x)
    # Don't allow the model to generate <unknown> tokens
    y = torch.nn.functional.softmax(y[:, -1, :y.shape[2]-1].view(-1).detach().cpu(), dim=0)
    if not sample:
        pred = y.argmax(dim=-1).view(-1)
    else:
        dist = torch.distributions.categorical.Categorical(probs=y)
        pred = dist.sample([1])
    next_word = pred.item()
    return next_word

def print_sentence(words):
    print(" ".join([index_to_word[word] for word in words]))

def generate_sentence(start=None, sample=False, length=50):
    if start == None:
        sentence = []
    else:
        words = start.split(" ")
        sentence = [word_to_index[x] for x in words]
    
    while len(sentence)<length:
        if len(sentence) < settings["max_input_length"]:
            next_word = generate_next_token(sentence, sample)
            sentence += [next_word]
        else:
            next_word = generate_next_token(sentence[-settings["max_input_length"]:], sample)
            sentence += [next_word]
    
    print_sentence(sentence)


In [108]:
transformer.train()
for i in range(5):
    generate_sentence("what is", False, 50)

what is the man ? servant . i am a gentleman . lafew . a gentleman , sir , a gentleman , a gentleman , a gentleman . clown . a gentleman , a knave , a gentleman , a gentleman . clown . a clown . clown . sir
what is your purpose ? first lord . i hope , my lord , i cannot tell you . cerimon . i am not , my lord , i am a gentleman of the forest , and i have a daughter to her , and i am a gentleman of
what is to me ? i am not able to be a fool , but i will not be a fool . i am a fool , and i will not be sworn to be a woman , and then ill be a woman . i will be sworn to
what is he ? second citizen . he is not a man , my lord . third citizen . he has a man , and a man of half a man . third citizen . he has a man , and he is a fool , and a man ,
what is he ? first citizen . he is a man of late , and he is not mad . second citizen . he is not hurt , sir , but hes a man . third citizen . hes a man , sir , i pray you , to be


In [105]:
transformer.train()
for i in range(5):
    generate_sentence("i", False, 50)

i am a gentleman of love , and a man of all the world . i am a gentleman of a child , and a tall gentleman . sir toby . a fellow , a woman , a woman , a woman , a woman , a woman , a
i am a gentleman , and a gentleman of a gentleman . sir toby . sir andrew , i have a letter to you . sir toby . i will be hangd to her , and i will not be sworn straight . sir toby . i will have it
i am a villain , and a villain , and a knave , a knave , a fool , a fool , a fool , a fool , a fool , a fool , a very knave . clown . a plague ! a fool , i faith , sir
i am a fool , and a soldier . parolles . i am a fool , sir , a man . lafew . i am a devil , sir , a devil , a horse , a rogue , a fool , a very melancholy , a woman , a
i am not mad . i am sorry to be sure of that i am not . i am sure you have no true . i am glad of it . i am sorry to have it . i am not of the duke , sir , but i am


In [112]:
transformer.train()
for i in range(5):
    generate_sentence("and", False, 50)

and the king . king henry . what , art thou dead ? king henry . ay , but what ? clarence . i am , my lord , and i am a king . king henry . i am sorry that i am , i have no subjects in
and the wind , the wind , the wind , the wind , the wind , the wind , the wind , the moon , the moon , the moon , the moon , the moon , the moon , the moon , the sky , the moon ; the
and the man that is the man . he is a man . i am sure he is a man , and he is a man of all , and i will be sure he is a man . if i be not , i will be sworn to be
and the rest . exeunt . scene iv . the same . enter the two or fourth lord scales . the king is dead . first lord . aside . i have been so much . second lord . aside . i am glad of him . i am not
and so i do . exit . scene ii . the same . enter a banquet . first citizen . well , sir , i will . fourth citizen . you are welcome , sir ; you shall have the cause . second citizen . you are the people .


Examples

- [if thou didst] not , thou art a fool . i am a fool , and thou art a fool . thou art a fool . clown . i am
- [i] have a woman , and i have a good heart . but i will be a man of mine , and i will not be satisfied .
- [i] am a man , and i am sure of it . i have said , and i will not be sworn to thee . i am a king ,
- [you are] a merciful old wench . i am not able to wait upon my friend . i am glad to have merry a launcelet , if i had a
- [you are] a beauteous blossom , and i will encounter thee with my nan . i am a proper friend . anne . i would not play a fool ,
- i am not drunk for certain , nor i am not of any part of my sex . i am sorry for neither
- [you are] in the field . enter a messenger . messenger . news , madam , news ! cleopatra . what news ? messenger . the news ? messenger . the king is gone . messenger . the lord mortimer is coming . antony . the noble antony is come
- [like the sky] , and the wind and the moon . o , what a one that is in the other ?
- [i] am a gentleman of love , and a man of all the world .